In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

tf.keras.backend.clear_session()

In [2]:
from src.models import base_model, cnn_model
from src import custom_losses, custom_metrics, optimizers
from src.data import data

Using TensorFlow backend.


In [3]:
from importlib import reload

In [4]:
paths = data.PATH()

In [5]:
dataset_path = f'{paths.PROCESSED_DATA_PATH}/REI-Dataset/'

In [6]:
batch_size = 64
n_classes = 6
epochs = 100
img_size = 128
n_channels = 3

In [7]:
model = cnn_model.CNN_Model(batch_size, n_classes, epochs, img_size, n_channels)

W1119 23:25:47.130082 140116871436096 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [8]:
train_generator, validation_generator = model.get_image_data_generator(dataset_path, train=True, validation=True)
weights = model.get_class_weights(train_generator.classes, model)
model.compile(loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)
#model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'],)

Found 4671 images belonging to 6 classes.
Found 562 images belonging to 6 classes.


In [9]:
model.show_summary()

##### CNN_Model #####
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 124, 64)      4864      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 58, 58, 128)       204928    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 256)       819456    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
flatten_1 (Flatten)          (No

In [10]:
steps_per_epoch, validation_steps = len(train_generator.classes), len(validation_generator.classes)

In [11]:
model.fit_from_generator(path=dataset_path,
                         train_steps_per_epoch=steps_per_epoch, validation_steps_per_epoch=validation_steps, 
                         train_generator=train_generator, validation_generator=validation_generator,
                         evaluate_net=True, use_model_check_point=False, use_early_stop=True, weighted=True)

W1119 23:26:04.134079 140116871436096 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification/src/models/base_model.py:363: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W1119 23:26:04.137207 140116871436096 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification/src/models/base_model.py:364: The name tf.local_variables_initializer is deprecated. Please use tf.compat.v1.local_variables_initializer instead.

W1119 23:26:04.139241 140116871436096 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification/src/models/base_model.py:365: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.



class_weights [1.31949153 1.24162679 0.61687797 1.09957627 0.973125   1.13815789]
**** Class weights ****
[1.31949153 1.24162679 0.61687797 1.09957627 0.973125   1.13815789]
*** ** *** *** *** ** ***


W1119 23:26:05.315100 140116871436096 deprecation.py:323] From /home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1119 23:26:07.232031 140116871436096 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/100
4671/4671 [==============================] - 1865s 399ms/step - loss: 0.1251 - categorical_accuracy: 0.9481 - val_loss: 1.3247 - val_categorical_accuracy: 0.7617
Epoch 2/100
4671/4671 [==============================] - 1636s 350ms/step - loss: 1.6970e-07 - categorical_accuracy: 1.0000 - val_loss: 3.9617 - val_categorical_accuracy: 0.7544
Epoch 3/100
4671/4671 [==============================] - 1411s 302ms/step - loss: 1.2040e-07 - categorical_accuracy: 1.0000 - val_loss: 2.2582 - val_categorical_accuracy: 0.7544
Epoch 4/100
4671/4671 [==============================] - 1414s 303ms/step - loss: 1.1925e-07 - categorical_accuracy: 1.0000 - val_loss: 2.4619 - val_categorical_accuracy: 0.7547
Epoch 5/100
4671/4671 [==============================] - 1410s 302ms/step - loss: 1.1921e-07 - categorical_accuracy: 1.0000 - val_loss: 2.9328 - val_categorical_accuracy: 0.7542
Epoch 6/100
4671/4671 [==============================] - 1408s 301ms/step - loss: 1.1921e-07 - categorical_accurac

In [ ]:
model.model_path

In [14]:
!ls '/home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification//models/MLP_Model/2019-11-16__18_00/'

weights-improvement-05-13.87.hdf5  weights-improvement-10-13.86.hdf5


In [ ]:
model.scores

In [ ]:
m = model.load_model(model.model_path)

In [ ]:
m.evaluate_from_generator(dataset_path)